# Notebook to run the V1 cortical column simulation and recreate figures from Billeh et al., 2020

In [ ]:
%pip install -qU "airavata-python-sdk[notebook]"
import airavata_jupyter_magic

%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud
%wait_for_runtime hpc_cpu --live
%switch_runtime hpc_cpu

In [ ]:
%%bash

git clone https://github.com/cyber-shuttle/V1_example workspace
ln -s $PWD/cybershuttle_data/allenai-v1/v1_point/network workspace/network
ln -s $PWD/cybershuttle_data/allenai-v1/v1_point/components workspace/components

In [ ]:
%cd workspace

In [ ]:
import matplotlib.pyplot as plt
import os
import pathlib

In [ ]:
filternet_config = 'filternet_configs/config.filternet.dg.2Hz.270deg.json'
pointnet_config = 'v1_point_config/config.json'

In [ ]:
#from run_filternet import run as run_filternet

#run_filternet(filternet_config)

In [ ]:
from run_pointnet import main as run_pointnet

run_pointnet(pointnet_config)

In [ ]:
import plotting_utils as pu

pointnet_config = 'output/config.json'
net = 'full'
sortby='tuning_angle'
radius = 400.0

plt.figure(figsize=(10, 6))


ax = pu.plot_raster(pointnet_config, sortby=sortby, **pu.settings[net])
ax.set_xlim([0, 2500])
plt.tight_layout()
config_folder = os.path.dirname(pointnet_config)
plt.savefig(f"{config_folder}/raster_by_{sortby}.png", dpi=300)

In [ ]:
spikes_df, hue_order, color_dict, layer_divisions = pu.make_figure_elements(pointnet_config, radius, sortby)

In [ ]:
# calculate firing rates
rates = spikes_df[spikes_df['timestamps']>500.0].groupby(['Sorted ID'])['timestamps'].count()/2.5
rates = rates.reset_index(name = 'Mean Rate')
rates_df = spikes_df[['population', 'Sorted ID', 'Cell Type', 'Tuning Angle', 'location']].drop_duplicates('Sorted ID')
rates_df = rates_df.merge(rates, how='left', left_on='Sorted ID', right_on = 'Sorted ID')


In [ ]:
import seaborn as sns
order = [1, 2, 4, 5, 6]
sns.boxplot(rates_df, x='location', y='Mean Rate', hue='Cell Type', order=order)

In [ ]:
from run_pointnet import main as run_pointnet

pointnet_config_movie = 'v1_point_config/config.movie.json'
run_pointnet(pointnet_config_movie)

In [ ]:
import plotting_utils as pu
pointnet_config_movie = 'output_movie/config.movie.json'
net = 'full'
sortby='tuning_angle'
radius = 400.0

plt.figure(figsize=(10, 6))


ax = pu.plot_raster(pointnet_config_movie, sortby=sortby, **pu.settings[net])
ax.set_xlim([0, 2500])
plt.tight_layout()
config_folder = os.path.dirname(pointnet_config_movie)
plt.savefig(f"{config_folder}/raster_by_{sortby}.png", dpi=300)

In [ ]:
pointnet_config_flash = 'v1_point_config/config.fullfield_250ms.json'
run_pointnet(pointnet_config_flash)

In [ ]:
pointnet_config_flash = 'output_flash/config.fullfield_250ms.json'
net = 'full'
sortby='tuning_angle'
radius = 400.0

plt.figure(figsize=(10, 6))


ax = pu.plot_raster(pointnet_config_flash, sortby=sortby, **pu.settings[net])
ax.set_xlim([0, 2500])
plt.tight_layout()
config_folder = os.path.dirname(pointnet_config_flash)
plt.savefig(f"{config_folder}/raster_by_{sortby}.png", dpi=300)

In [ ]:
%stop_runtime hpc_cpu
%switch_runtime local